#Notebook
- **Professor:** Iális Cavalcante
- **Monitor:** Iago Magalhães
- **Disciplina:** Ciência de dados
- **Curso:** Engenharia da Computação
- **Descrição:** PCA, ou análise de componentes principais é uma técnica para reduzir a dimensão de um conjunto de dados preservando, de certa forma, suas propriedades. Executar essa técnica na mão ou sem ajuda de funções prontas é um bom exercício para aprender a fundo o que está por trás desse processo. Nesse tutorial faremos um exemplo de PCA na mão, sempre conferindo as contas com Python e, depois, faremos o mesmo exemplo usando a biblioteca Scikit-learn.

##Importações

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

##Analise de dados

In [ ]:
df = pd.DataFrame({
    'var1':[1,3,1,2,2,1],
    'var2':[2,4,3,4,3,4],
    'target': [0,1,0,1,1,1]
})
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
mean_vec = np.mean(X, axis=0)
print(mean_vec)

In [ ]:
M = X - mean_vec

###Calculando a Matriz de Covariância

In [ ]:
C = M.T.dot(M) / (X.shape[0]-1)
C

###Calculando Autovalores e Autovetores

In [ ]:
# determinando auto-valores e auto-vetores
autovalores, autovetores = np.linalg.eig(C)

# imprimindo auto-valores de C
print("Auto-valores:")
print(autovalores)
print()

# imprimindo autovetores
print("Auto-vetores:")
print(autovetores)

In [ ]:
pares_de_autos = [
    (
        np.abs(autovalores[i]),
        autovetores[:,i]
    ) for i in range(len(autovalores))
]
pares_de_autos.sort()
pares_de_autos.reverse()

In [ ]:
# calculando a variância explicada e a variância explicada cumulativa
total = sum(autovalores)
var_exp = [
    (i / total)*100 for i in sorted(
        autovalores, reverse=True
    )
]
cum_var_exp = np.cumsum(var_exp)

In [ ]:
# visualizando as informações
x = [
    'PC %s' %i for i in range(
        1,len(autovalores)+1
    )
]
df_temp = pd.DataFrame(
    {'auto-valores': autovalores,
     'cum_var_exp':cum_var_exp,
     'var_exp':var_exp,
     'Componente':x}
)
print(df_temp)
print()
print("Auto-vetores")
for autovetor in [p[1] for p in pares_de_autos]:
    print(autovetor)
print()

In [ ]:
# visualizando graficamente os dados originais
sns.pairplot(
    df,
    vars = ['var1','var2'],
    hue='target',
    diag_kind="hist"
)
plt.show()

In [ ]:
# visualizando graficamente os dados através da mudança de base
# considerando base de auto-vetores
n_componentes = 2 # projetamos nas 2 (duas) primeiras componentes
autovetores = [p[1] for p in pares_de_autos]
A = autovetores[0:n_componentes]
X = np.dot(X,np.array(A).T)
new_df = pd.DataFrame(X,columns=['pc1','pc2'])
new_df['target'] = df['target']
sns.pairplot(
    new_df, vars = ['pc1','pc2'], hue='target', diag_kind="hist"
)
plt.show()

##Aplicando técnicas de redução de dimensionalidade em imagens

Outra aplicação é a redução de dimensão em imagens. Para esse exemplo, escolhemos a imagem Pássaro mineiro barulhento retirada do site Pixabay.
Link imagem: https://leandrocl2005.github.io/pca_na_mao_e_no_python/bird.jpg

Carregando a imagem

In [ ]:
X = plt.imread('bird.jpg')
X.shape

Observe que a matriz tem 1920 linhas e 1899 colunas. Aplicaremos, em seguida, a técnica PCA.

In [ ]:
pca = PCA(0.99) # variância explicada de 0.99
lower_dimension_data = pca.fit_transform(X)
lower_dimension_data.shape

O número 0.99 passado como parâmetro para o métodos PCA diz que queremos um número de componentes que nos garanta 99% de variância explicada cumulativa. Observe que para isso, utilizou-se 145 componentes. Isso é uma redução imensa, uma vez que a original possuia 1899 colunas. Queremos ver, caso necessário recuperar a imagem, como essas imagens ficam com a dimensão reduzida. Para isso serão necessários duas funções auxiliares.

In [ ]:
def pca_with_var_exp(X, var_exp=0.99):
    pca = PCA(var_exp) # variância explicada de 0.99
    lower_dimension_data = pca.fit_transform(X)
    print(lower_dimension_data.shape)
    approximation = pca.inverse_transform(lower_dimension_data)
    return approximation

def plot_subplot(X, i):
    plt.subplot(3,2,i)
    plt.imshow(X, cmap="gray")
    plt.xticks([])
    plt.yticks([])

A primeira função reduz a imagem X mantendo var_exp de variância explicada cumulativa. Compararemos 3 valores: 0.99, 0.95 e 0.90.

In [ ]:
# calculando algumas aproximações
img_1 = pca_with_var_exp(X, var_exp=0.99)
img_2 = pca_with_var_exp(X, var_exp=0.95)
img_3 = pca_with_var_exp(X, var_exp=0.90)

A segunda função mostra o gráfico. Utilizaremos ela para comparar as imagens obtidas com a original.

In [ ]:
# visualizando aproximações
plt.figure(figsize=(6,8))

plot_subplot(X, 1)
plt.title("Original")
plot_subplot(img_1, 2)
plt.title("Var. Explicada de 99%")
plot_subplot(X, 3)
plt.title("Original")
plot_subplot(img_2, 4)
plt.title("Var. Explicada de 95%")
plot_subplot(X, 5)
plt.title("Original")
plot_subplot(img_3, 6)
plt.title("Var. Explicada de 90%")

plt.tight_layout()
plt.show()

##Referências
- [PCA na mão e no Python](https://leandrocruvinel.medium.com/pca-na-m%C3%A3o-e-no-python-d559e9c8f053)